In [1]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
#Read data from file on computer

df = pd.read_json(r'D:\ITESM\Essex\Assignment1\spider\spider\train_spider.json')

df.shape

(7000, 7)

In [3]:
from keras.models import Model
from keras.layers import Input, LSTM,   Dense